In [3]:
import nltk 
import re
import math
import pandas as pd
import numpy as np
import unicodedata

from collections import Counter
from unicodedata import normalize
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz


#Regex para encontrar tokens
REGEX_WORD = re.compile(r'\w+')
#Numero de tokens em sequencia
N_GRAM_TOKEN = 3

#Faz a normalizacao do texto e remove espacos, numeros, pontuação e tirando acentos

def chr_remove(old, to_remove):
    new_string = old
    for x in to_remove:
        new_string = new_string.replace(x, '')
    return new_string

def text_normalizer(src):
    texto = re.sub('\s+', ' ',
                normalize('NFKD', src)
                   .encode('ASCII','ignore')
                   .decode('ASCII')
           ).lower().strip()
    query = texto
    stopwords = ['vinho','tinto','branco','rose','espumante','champagne','cada','caixa','caixas','cabernet','sauvignon',
                'pinot','carmenere','merlot','noir','garrafa','garrafas','seco','brut','extra','malbec','syrah','shiraz',
                 'tannat','fino','contendo','mesa,','com','safra','unidades','blanco','de','acondicionado','em','graduacao','%','embalado','embalados',
                 'lote:','safra:','-','ml.','alcoolica','grad','acondicionados','meio','mesa','vol','aop','igp','aoc','ml',
                 'total','totalizando','sendo','rosé','vin','fino,','cxs','acima','de','para','com','teor','red',
                 'em','white','rouge','marca','tempranillo','cava','crianza','rioja','vino','reserva','reservado','dop',
                 'vinhos','alc','garnacha','pinotage','seco','douro','ref','uva','uvas','proprio','alentejo','peninsula',
                 'humano','dispostos','marca','regional','cxs','teor','frisante','frizzante','igt','vino','italiano',
                 'bianco','rosso','comercial','alcoolico','del','suave','bianco',
                 
                 ]
    carac = chr_remove(query,",_-%.&/\$#@!?*)(:;'+=")
    numbers = re.sub('[0-9]+', '', carac)
    querywords = numbers.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result
    

#Faz o calculo de similaridade baseada no coseno
def cosine_similarity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        coef = float(numerator) / denominator
        if coef > 1:
            coef = 1
        return coef

#Monta o vetor de frequencia da sentenca
def sentence_to_vector(text, use_text_bigram):
    words = REGEX_WORD.findall(text)
    accumulator = []
    for n in range(1,N_GRAM_TOKEN):
        gramas = ngrams(words, n)
        for grama in gramas:
            accumulator.append(str(grama))

    if use_text_bigram:
        pairs = get_text_bigrams(text)
        for pair in pairs:
            accumulator.append(pair)

    return Counter(accumulator)

#Obtem a similaridade entre duas sentencas
def get_sentence_similarity(sentence1, sentence2, use_text_bigram=False):
    vector1 = sentence_to_vector(text_normalizer(sentence1), use_text_bigram)
    vector2 = sentence_to_vector(text_normalizer(sentence2), use_text_bigram)
    return cosine_similarity(vector1, vector2)

#Metodo de gerar bigramas de uma string
def get_text_bigrams(src):
    s = src.lower()
    return [s[i:i+2] for i in range(len(s) - 1)]

if __name__ == "__main__":
    w1 = 'VINHO BRANCO FINO DE MESA, CHABLIS 1ER CRU MONT DE MILIEU, SAFRA: 2015, ACONDICIONADAS EM 10 CAIXAS CONTENDO 12 GARRAFAS DE 750 ML CADA.'
    words = [
        'Vinho Fino de Mesa Seco Tinto - H.O. Sousão Tinto 2014, teor alcoólico 13,5%, lote 18416 - acondicionado em 20 caixas com 3 garrafas de 750ml cada - HOSOU',
        'Vinho Fino de Mesa Seco Tinto - H.O. Achado Tinto 2013, teor alcoólico 13,5%, lote 17708 - acondicionado em 90 caixas com 6 garrafas de 750ml cada - HOAT ',
        'VINHO BRANCO FINO DE MESA, CHABLIS 1ER CRU VAILLONS, SAFRA: 2015, ACONDICIONADAS EM 10 CAIXAS CONTENDO 12 GARRAFAS DE 750 ML CADA. '
        
    ]

    print('Busca: ' + w1)

    #Nivel de aceite (75%)
    cutoff = 0.75
    #Sentenças similares
    result = []

    for w2 in words:
        print('\nCosine Sentence --- ' + w2)

        #Calculo usando similaridade do coseno com apenas tokens
        similarity_sentence = get_sentence_similarity(w1, w2)
        print('\tSimilarity sentence: ' + str(similarity_sentence))

        #Calculo usando similaridade do coseno com tokens e com ngramas do texto
        similarity_sentence_text_bigram = get_sentence_similarity(w1, w2, use_text_bigram=True)
        print('\tSimilarity sentence: ' + str(similarity_sentence_text_bigram))

        if similarity_sentence >= cutoff:
            result.append((w2, similarity_sentence))

    print('\nResultado mais similar:')
    #Exibe resultados
    for data in result:
        print(data)

Busca: VINHO BRANCO FINO DE MESA, CHABLIS 1ER CRU MONT DE MILIEU, SAFRA: 2015, ACONDICIONADAS EM 10 CAIXAS CONTENDO 12 GARRAFAS DE 750 ML CADA.

Cosine Sentence --- Vinho Fino de Mesa Seco Tinto - H.O. Sousão Tinto 2014, teor alcoólico 13,5%, lote 18416 - acondicionado em 20 caixas com 3 garrafas de 750ml cada - HOSOU
	Similarity sentence: 0.0
	Similarity sentence: 0.0

Cosine Sentence --- Vinho Fino de Mesa Seco Tinto - H.O. Achado Tinto 2013, teor alcoólico 13,5%, lote 17708 - acondicionado em 90 caixas com 6 garrafas de 750ml cada - HOAT 
	Similarity sentence: 0.0
	Similarity sentence: 0.11697706772393274

Cosine Sentence --- VINHO BRANCO FINO DE MESA, CHABLIS 1ER CRU VAILLONS, SAFRA: 2015, ACONDICIONADAS EM 10 CAIXAS CONTENDO 12 GARRAFAS DE 750 ML CADA. 
	Similarity sentence: 0.6030226891555273
	Similarity sentence: 0.7715167498104595

Resultado mais similar:


In [7]:
def get_dataframe_similarity(comparer, finder, cutoff):
    print('cutoff= ' + str(cutoff))
    result = []
    comparer = np.array(comparer)
    for find in np.array(finder):
        max_coef = 0
        data = find
        for compare in comparer:
            similarity = get_sentence_similarity(find[0], compare[0])
            if similarity >= cutoff:
                if similarity > max_coef:
                    #print('Trocando Marca:' + data[1] + ' por Marca: ' + compare[1])
                    print(data[0] + ' ---- ' + compare[0] + ' - similaridade: ' + str(float( '%g' % ( similarity * 100 ) )) + '%')
                   
                    data[1] = compare[1]
                    data[2] = compare [2]
                    data[3] = compare [3]
                    data[4] = compare [4]
                    max_coef = similarity
        result.append(data)

    result = np.array(result)
    dataFrame = pd.DataFrame()
    dataFrame['Descr. Detalhada'] = result[..., 0]
    dataFrame['Fabricante Produtor'] = result[..., 1]
    dataFrame['Distribuidor']= result[..., 2]
    dataFrame['Marca']= result[..., 3]
    dataFrame['Rotulo']= result[..., 4]
    return dataFrame

In [8]:
X1 = pd.read_csv('africa1.csv', sep=';', encoding="latin1", dtype='unicode')

In [9]:
X1.head(5)

,Descr. Detalhada,Fabricante Produtor,Distribuidor,Marca,Rotulo
0,700 CAIXAS - VINHO FINO BRANCO MEIO SECO CHARD...,x,x,x,x
1,700 CAIXAS - VINHO FINO TINTO MEIO SECO SHIRAZ...,x,x,x,x
2,VINHO FINO DE MESA - 745 CAIXAS C/ KUMALA CABE...,x,x,x,x
3,VINHO FINO DE MESA - 910 CAIXAS C/ KUMALA MERL...,x,x,x,x
4,VINHO FINO DE MESA - 345 CAIXAS C/ KUMALA COLO...,x,x,x,x


In [10]:
X2 = pd.read_csv('africa2.csv', sep=';', encoding="latin1")

In [11]:
X2.head(5)

,Descr. Detalhada,Fabricante Produtor,Distribuidor,Marca,Rotulo
0,"200 CXS - VINHO FINO BRANCO SECO CHENIN BLANC,...",NIEL JOUBERT WINES,BWC BRASIL COMERCIO E IMPORTACAO-BERKMANN WINE...,KLEINKLOOF,KLEINKLOOF
1,"200 CXS - VINHO FINO TINTO SECO , CABERNET SAU...",NIEL JOUBERT WINES,BWC BRASIL COMERCIO E IMPORTACAO-BERKMANN WINE...,KLEINKLOOF MOUNTAIN,KLEINKLOOF MOUNTAIN
2,200 CXS - VINHO FINO BRANCO SECO SAUVIGNON BLA...,NIEL JOUBERT WINES,BWC BRASIL COMERCIO E IMPORTACAO-BERKMANN WINE...,KLEINKLOOF,KLEINKLOOF
3,VINHO TINTO FINO MEIO SECO NEDERBURG FOUNDATIO...,DISTELL LIMITED,CASA FLORA,NEDERBURG FOUNDATION,NEDERBURG FOUNDATION
4,VINHO TINTO FINO MEIO SECO NEDERBURG 1791 PINO...,DISTELL LIMITED,CASA FLORA,NEDERBURG 1971,NEDERBURG 1791


In [142]:
h= 'oi'
print (h)

oi


In [12]:
if __name__ == "__main__":
    cutoff = 0.77
    dataFrame1 = X1
    

    dataFrame2 = X2
    

    dataResult = get_dataframe_similarity(comparer=dataFrame2, finder=dataFrame1, cutoff=cutoff)
    print(dataResult)

cutoff= 0.77
700 CAIXAS - VINHO FINO BRANCO MEIO SECO CHARDONNAY / SAUVIGNON BLANC, SAFRA 2017 - MARCA GOIYA CHARDONNAY SAUV.BLANC, EM CXS C/06 GARRAFAS DE 750ML  ---- 180 CAIXAS - VINHO FINO BRANCO MEIO SECO CHARDONNAY / SAUVIGNON BLANC, SAFRA 2017 - MARCA GOIYA CHARDONNAY SAUV.BLANC, EM CXS C/06 GARRAFAS DE 750ML - LOTE L7 136                                                                                         - similaridade: 87.4475%
700 CAIXAS - VINHO FINO BRANCO MEIO SECO CHARDONNAY / SAUVIGNON BLANC, SAFRA 2017 - MARCA GOIYA CHARDONNAY SAUV.BLANC, EM CXS C/06 GARRAFAS DE 750ML  ---- 700 CAIXAS - VINHO FINO BRANCO MEIO SECO CHARDONNAY / SAUVIGNON BLANC, SAFRA 2017 - MARCA GOIYA CHARDONNAY SAUV.BLANC, EM CXS C/06 GARRAFAS DE 750ML  - similaridade: 100.0%
700 CAIXAS - VINHO FINO TINTO MEIO SECO SHIRAZ / PINOTAGE, SAFRA 2016 - MARCA GOIYA SHIRAZ PINOTAGE, EM CXS C/06 GARRAFAS DE 750ML  ---- 700 CAIXAS - VINHO FINO TINTO MEIO SECO SHIRAZ / PINOTAGE, SAFRA 2016 - MARCA GOIYA SHIRAZ 

SWWCSZ16-6SW - 227029 - VINHO TINTO DE MESA SECO FINO, SWARTLAND WINEMAKERS SHIRAZ COLLECTION, SAFRA 2016 - ACONDICIONADOS EM CAIXAS C/ 06 GARRAFAS DE 750ML  ---- SWWCOI16-6SW - 227028 - VINHO TINTO DE MESA SECO FINO, SWARTLAND WINEMAKERS PINOTAGE COLLECTION, SAFRA 2016 - ACONDICIONADOS EM CAIXAS C/ 06 GARRAFAS DE 750ML  - similaridade: 77.7778%
SWWCSZ16-6SW - 227029 - VINHO TINTO DE MESA SECO FINO, SWARTLAND WINEMAKERS SHIRAZ COLLECTION, SAFRA 2016 - ACONDICIONADOS EM CAIXAS C/ 06 GARRAFAS DE 750ML  ---- SWWCSZ16-6SW - 227029 - VINHO TINTO DE MESA SECO FINO, SWARTLAND WINEMAKERS SHIRAZ COLLECTION, SAFRA 2016 - ACONDICIONADOS EM CAIXAS C/ 06 GARRAFAS DE 750ML  - similaridade: 100.0%
234958 - VINHO TINTO DE MESA SECO FINO, SARONSBERG PROVENANCE SHIRAZ, SAFRA 2015 - ACONDICIONADOS EM 20 CAIXAS C/ 06 GARRAFAS DE 750ML  ---- VINHO TINTO DE MESA SECO FINO, SARONSBERG PROVENANCE SHIRAZ, SAFRA 2014, ACONDICIONADOS EM 100 CAIXAS C/ 06 GARRAFAS DE 750ML.  - similaridade: 100.0%
234959 - VINHO T

GARRAFAS DE 750 ML - SAFRA 2017 , VINHO FINO DE MESA, TINTO, BARISTA PINOTAGE 17, TEOR ALCOOLICO: 13,5 %  ---- GARRAFAS DE 750 ML - SAFRA 2017 , VINHO FINO DE MESA, TINTO, BARISTA PINOTAGE 17, TEOR ALCOOLICO: 13,5 %  - similaridade: 100.0%
GARRAFAS DE 750 ML - SAFRA 2015 , VINHO FINO DE MESA, BRANCO, BOEKENHOUTSKLOOF SEMILLON 15, TEOR ALCOOLICO: 13,5 %  ---- GARRAFAS DE 750 ML - SAFRA 2015 , VINHO FINO DE MESA, BRANCO, BOEKENHOUTSKLOOF SEMILLON 15, TEOR ALCOOLICO: 13,5 %  - similaridade: 100.0%
GARRAFAS DE 750 ML - SAFRA 2015 , VINHO FINO DE MESA, TINTO, PORSELEINBERG SYRAH 15, TEOR ALCOOLICO: 13,5 %  ---- GARRAFAS DE 750 ML - SAFRA 2015 , VINHO FINO DE MESA, TINTO, PORSELEINBERG SYRAH 15, TEOR ALCOOLICO: 13,5 %  - similaridade: 100.0%
GARRAFAS DE 750 ML - SAFRA 2016 , VINHO FINO DE MESA, TINTO, WOLFTRAP 2016, TEOR ALCOOLICO: 14,5 % VINHO TIPICO E TRADICIONAL NO PAIS DE ORIGEM  ---- GARRAFAS DE 750 ML - SAFRA 2015 , VINHO FINO DE MESA, TINTO, KANONKOP PINOTAGE 15, TEOR ALCOOLICO: 14,5 

Vinho Tinto, Seco, Namaqua Pinotage, Marca Namaqua, Garrafa de 750 ML, Graduação 13,5%, Safra 2017, Lote L250, caixas com 06 garrafas.  ---- VINHO BRANCO, SECO, NAMAQUA SAUVIGNON BLANC, MARCA NAMAQUA, GARRAFA DE 750 ML, GRADUAÇÃO 13,5%, SAFRA 2016, LOTE L6111, CAIXAS COM 06 GARRAFAS. - similaridade: 80.403%
Vinho Tinto, Seco, Namaqua Pinotage, Marca Namaqua, Garrafa de 750 ML, Graduação 13,5%, Safra 2017, Lote L250, caixas com 06 garrafas.  ---- VINHO TINTO, MEIO SECO, NAMAQUA CABERNET SAUVIGNON, MARCA NAMAQUA, GARRAFA DE 750 ML, GRADUAÇÃO 13,5%, SAFRA 2015, LOTE L6325, CAIXAS COM 06 GARRAFAS. - similaridade: 100.0%
Vinho Tinto, Seco, Namaqua Shiraz, Marca Namaqua, Garrafa de 750 ML, Graduação 13,5%, Safra 2016, Lote L206, caixas com 06 garrafas.  ---- VINHO BRANCO, SECO, NAMAQUA SAUVIGNON BLANC, MARCA NAMAQUA, GARRAFA DE 750 ML, GRADUAÇÃO 13,5%, SAFRA 2016, LOTE L6111, CAIXAS COM 06 GARRAFAS. - similaridade: 80.403%
Vinho Tinto, Seco, Namaqua Shiraz, Marca Namaqua, Garrafa de 750 ML, 

610978 - VINHO FINO TINTO SECO, NOME COMERCIAL: VINO TINTO WINEMASTERS PINOTAGE , MARCA: NEDERBURG, SENDO 6 GARRAFAS DE 750ML EM CADA CAIXA , GRADUAÇÃO ALCOÓLICA: 14%, SAFRA: 2016  ---- 610978 - VINHO FINO TINTO SECO, NOME COMERCIAL: VINO TINTO WINEMASTERS PINOTAGE , MARCA: NEDERBURG, SENDO GARRAFAS DE 750ML EM CADA CAIXA , GRADUAÇÃO ALCOÓLICA: 14%, SAFRA: 2015 - similaridade: 100.0%
170 CXS - VINHO FINO BRANCO SECO DRY CHENIN BLANC, MARCA KLEINKLOOF, SAFRA 2017, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  ---- 200 CXS - VINHO FINO BRANCO SECO CHENIN BLANC, MARCA KLEINKLOOF, SAFRA 2016, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA. - similaridade: 84.5154%
170 CXS - VINHO FINO BRANCO SECO DRY CHENIN BLANC, MARCA KLEINKLOOF, SAFRA 2017, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  ---- 250 CXS - VINHO FINO BRANCO SECO DRY CHENIN BLANC, MARCA KLEINKLOOF, SAFRA 2017, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  - similaridade: 100.0%
VINHO FINO BRANCO - THE BEACH HOUSE SAUVIGNON BLANC 2017 - TEOR ALCO

280 CXS - VINHO TINTO SECO, MARCA PEARLY BAY, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  ---- 300 CXS - VINHO FINO TINTO SECO, MARCA PEARLY BAY DRY RED, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  - similaridade: 77.4597%
280 CXS - VINHO TINTO SECO, MARCA PEARLY BAY, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  ---- 280 CXS - VINHO TINTO SECO, MARCA PEARLY BAY, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  - similaridade: 100.0%
560 CXS - VINHO BRANCO SECO, MARCA PEARLY BAY, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  ---- 300 CXS - VINHO FINO TINTO SECO, MARCA PEARLY BAY DRY RED, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  - similaridade: 77.4597%
560 CXS - VINHO BRANCO SECO, MARCA PEARLY BAY, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  ---- 280 CXS - VINHO TINTO SECO, MARCA PEARLY BAY, EM CAIXAS COM 06 GARRAFAS DE 750ML CADA.  - similaridade: 100.0%
VINHO TINTO MEIO SECO W.O. ROBERTSON/ SOUTH AFRICA CRANE RED MERLOT, ACONDICIONADO EM 200 CAIXAS CONTENDO 6 GARRAFAS DE 750ML CADA MARCA: GREAT EXPECTAT

VINHO BRANCO SECO W.O. STELLENBOSCH/ SOUTH AFRICA VIOGNIER E VERDELHO ACONDICIONADO EM 40 CAIXAS CONTENDO 6 GARRAFAS DE 750ML CADA MARCA: VENDETTA - SAFRA 2016 - GRADUACAO ALCOOLICA 12,5% VOL - LOTE L2016/31189.  ---- VINHO BRANCO SECO W.O. STELLENBOSCH/ SOUTH AFRICA VIOGNIER E VERDELHO ACONDICIONADO EM 40 CAIXAS CONTENDO 6 GARRAFAS DE 750ML CADA MARCA: VENDETTA - SAFRA 2016 - GRADUACAO ALCOOLICA 12,5% VOL - LOTE L2016/31189.  - similaridade: 100.0%
VINHO BRANCO SECO W.O. STELLENBOSCH/ SOUTH AFRICA CHENIN BLANC, CHARDONNAY E VERDELHO ACONDICIONADO EM 40 CAIXAS CONTENDO 6 GARRAFAS DE 750ML CADA MARCA: CREMELLO - SAFRA 2015 - GRADUACAO ALCOOLICA 13,5% VOL - LOTE L2016/21477.  ---- VINHO BRANCO SECO W.O. STELLENBOSCH/ SOUTH AFRICA CHENIN BLANC, CHARDONNAY E VERDELHO ACONDICIONADO EM 40 CAIXAS CONTENDO 6 GARRAFAS DE 750ML CADA MARCA: CREMELLO - SAFRA 2015 - GRADUACAO ALCOOLICA 13,5% VOL - LOTE L2016/21477.  - similaridade: 100.0%
VINHO TINTO SECO W.O. STELLENBOSCH/ SOUTH AFRICA SHIRAZ E GR

TypeError: normalize() argument 2 must be str, not float

In [ ]:
import numpy as np

In [ ]:
writer = pd.ExcelWriter('AFRICA.xlsx', engine='xlsxwriter')

In [ ]:
dataResult.to_excel(writer, sheet_name='tabulacao')

In [ ]:
writer.save()